# References
https://medium.com/rate-engineering/hierarchical-text-classification-for-rates-404c6c399f6b

https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python

https://stackoverflow.com/questions/40077432/why-is-scikit-learn-svm-svc-extremely-slow

https://scikit-learn.org/0.18/auto_examples/hetero_feature_union.html

https://scikit-learn.org/stable/modules/feature_extraction.html
# Connect to Shared Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

# Note that we don't use '!' here, so we can use IPython's builtin cd utility. Else we'd be running cd in a subshell, which is pointless.
%cd "/content/drive/Shareddrives/Thesis/Data"

# Check if we can see our data
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/Shareddrives/Thesis/Data
a.dot
All_Amazon_100k_root_svm_title_desc.pkl
All_Amazon_Meta_chunks
All_Amazon_r375k_hierarchical_title_desc.pkl
All_Amazon_r750k_desc_cleaned.feather
All_Amazon_r750k_empty_dropped_cat_cleaned.feather
All_Amazon_r750k_empty_dropped_nonstrip.feather
All_Amazon_r750k_trimmed_cat.feather
ama_cat.jpeg
ama_cat.png
Amazon_Arts_Crafts_and_Sewing
Amazon_Books
Amazon_Clothing_Shoes_and_Jewelry
Amazon_Electronics
Amazon_Grocery_and_Gourmet_Food
Amazon_Industrial_and_Scientific
Amazon_Marketing
Amazon_Musical_Instruments
Amazon_Pet_Supplies
a.png
a.txt
Electronics_tfidf_sgd_leaf.pkl
graph.png
graph.txt
MetaDatas_Unzipped
OutputFile.png
Pet_Supplies.hdf5
Smol_Amazon
test-output
Walmart_Marketing


# Import common libraries

In [ ]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from random import shuffle

!pip install swifter
import swifter

# Notebook settings


In [ ]:
### Dataset configuration
# The parquet folder. It should be located inside datasets/.
DATASET_NAME   = 'Electronics.parquet'
# The input text column 
FEATURE_COLS = ['title']
### Checkpoint configuration
# Whether to train from scratch or to load a checkpoint
# Checkpoint iteration to load if not training from scratch
LOAD_ITERATION=0
# Last or best results from that iteration?
LOAD_BEST=True

### System configuration
# Will try to use your NVIDIA GPU if one is available. Set to False to force CPU computation
PREFER_GPU         = True
# If you don't have the huggingface transformers library installed, flip this to True.
# You only need to do this once. Once DistilBERT has been downloaded, it will be cached in your system's default user cache folder.
# Once it is cached, please set this to False to avoid redownloads.
INSTALL_DISTILBERT = False

# Load cleaned data

In [ ]:
data = dd.read_parquet('../../datasets/{}'.format(DATASET_NAME))
with pd.option_context('display.max_colwidth', None):
    print (data.iloc[0])

category                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           [Camera & Photo, Video Surveillance, Surveillance Systems, Surveillance DVR Kits]
title                                                                                                                                                                                              

## Column selector
Selects a column from the dataset for processing by a sub-pipeline.

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
  def __init__(self, key, verbose=False):
    self.key = key
    self.verbose = verbose

  def fit(self, x, y=None):
    return self

  def transform(self, df):
    if self.verbose:
      print('Selecting column',self.key)
    return df[self.key]

# Text column stemmer
Stems a text column (can be p
assed as a Pandas series). Currently only English is supported (as we need to import more languages manually).

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

# These can't be put inside the class since they don't have _unload(), which prevents
# joblib from correctly parallelising the class if included.
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

class ColumnStemmer(BaseEstimator, TransformerMixin):
  def __init__(self, verbose=False):
    self.verbose = verbose

  def stem_and_concat(self, text):
    words = word_tokenize(text)
    result_list = map(lambda word: stemmer.stem(word) if word not in stop_words else word, words)
    return ' '.join(result_list)

  def fit(self, x, y=None):
    return self

  def transform(self, series):
    if self.verbose:
      print('Stemming column', series.name)
    return series.swifter.apply(self.stem_and_concat)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Shuffling and splitting dataset
80% training, 10% validation, 10% test. Validation set isn't really used here, but we still split out to it anyway to ensure with the same seed, we get the exact same training and test sets so we could compare results with other newer models. 

In [ ]:
FULL_SET = True
TRAIN_SET_RATIO = 0.8
VAL_SET_RATIO = 0.1
# The rest is test set
# Don't change this if you want a consistent sampling for easier comparisons
RANDOM_SEED = 123
small_data = None
if not FULL_SET:
    small_data = data.sample(frac = 0.25, random_state=RANDOM_SEED)

In [ ]:
train_set = None
test_set = None
if FULL_SET:
    train_set = data.sample(frac = TRAIN_SET_RATIO, random_state=RANDOM_SEED)
    val_test_set = data.drop(train_set.index)
else:
    train_set = small_data.sample(frac = TRAIN_SET_RATIO, random_state=RANDOM_SEED)
    val_test_set = small_data.drop(train_set.index)

val_set = val_test_set.sample(frac = VAL_SET_RATIO / (1-TRAIN_SET_RATIO), random_state=RANDOM_SEED)
test_set = val_test_set.drop(val_set.index)

train_set = train_set.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

(481622, 5)
(60203, 5)
(60202, 5)


In [ ]:
print(train_set['title'])

0         iPad Air 2 Screen Protector, BoxWave® [ClearTo...
1                           L3) 18-200MM DC 3.5-6.3 F/NIKON
2         SSIMOO Polka Dot Briefcase Handbag Sleeve Bag ...
3         Precision FRD03-AUX/S PIE Auxiliary Input Conv...
4                                 Q-STOR - USB To PS2 Cable
                                ...                        
481617    Honeywell 6000-USB-1 USB Communication & Charg...
481618    OPS DIS V57 SSS V41 Diagnose and Programming P...
481619    Canon VIXIA HF R500 Camcorder Memory Card 64GB...
481620    120w 19v 6.3A Replacement Ac Adapter Toshiba f...
481621    PAG Multi-functional Laptop and Reading Bamboo...
Name: title, Length: 481622, dtype: object


# Extract category level n into Y
For training against the rest of the columns. Here we shouldn't train against the root level, since all 100k samples have "Clothing, Shoes & Jewelry" as their root category.

**subject to change once we have cleaned up those bigger randomised slices**

In [ ]:
# 0-4 where 0 is root
LEVEL = 1
# For now, only train against title and description
X_train = train_set[FEATURE_COLS]
X_test = test_set[FEATURE_COLS]
y_train = train_set['category'].swifter.apply(lambda row: row[LEVEL])
y_test = test_set['category'].swifter.apply(lambda row: row[LEVEL])

# Check dimensionalities
print(X_train.values.shape)
print(y_train.values.shape)

Pandas Apply:   0%|          | 0/481622 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/60202 [00:00<?, ?it/s]

(481622, 1)
(481622,)


# Classifier construction
## Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from tempfile import mkdtemp
cachedir = mkdtemp()
# Note to dumb future self: clf stands for classifier in scikit-learn parlance
def getPipeline(clf):
  return Pipeline([
    # Extract title and description
    ('selector', ColumnSelector(key='title', verbose=True)),
    ('stemmer', ColumnStemmer(verbose=True)),
    ('tfidf', TfidfVectorizer(min_df=50)),

    # Use a SVC classifier on the combined features
    ('clf', clf),
  ])
    

## Init classifier instance & train
Currently using SGD.

In [ ]:
from sklearn import linear_model
from sklearn import svm
import joblib
from sklearn.model_selection import GridSearchCV
import time

clf_sgd = getPipeline(linear_model.SGDClassifier(loss='modified_huber',verbose=True, max_iter=1000))
param_grid_sgd = {} 

sgd_search = GridSearchCV(estimator=clf_sgd, param_grid=param_grid_sgd, n_jobs=-1)

start = time.time()
sgd_search.fit(X_train, y_train)

print(sgd_search.best_score_)
print(sgd_search.best_params_)
end=time.time()

folder_name = 'checkpoints-' + DATASET_NAME
!mkdir $folder_name
PICKLE_PATH = './{}/current.pkl'.format(folder_name)

print('First level took', end - start, 'ms')
joblib.dump(sgd_search, PICKLE_PATH)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  

Selecting column title
Stemming column title


Pandas Apply:   0%|          | 0/481622 [00:00<?, ?it/s]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 1.68, NNZs: 2007, Bias: -1.304081, T: 481622, Avg. loss: 0.001857
Total training time: 0.18 seconds.
-- Epoch 2
Norm: 1.65, NNZs: 2345, Bias: -1.259032, T: 963244, Avg. loss: 0.001619
Total training time: 0.36 seconds.
-- Epoch 3
Norm: 1.61, NNZs: 2557, Bias: -1.236510, T: 1444866, Avg. loss: 0.001590
Total training time: 0.53 seconds.
-- Epoch 4
Norm: 1.56, NNZs: 2644, Bias: -1.221848, T: 1926488, Avg. loss: 0.001580
Total training time: 0.71 seconds.
-- Epoch 5
Norm: 1.50, NNZs: 2756, Bias: -1.212467, T: 2408110, Avg. loss: 0.001571
Total training time: 0.88 seconds.
-- Epoch 6
Norm: 1.48, NNZs: 2797, Bias: -1.203462, T: 2889732, Avg. loss: 0.001572
Total training time: 1.05 seconds.
Convergence after 6 epochs took 1.05 seconds
-- Epoch 1
Norm: 18.52, NNZs: 8232, Bias: -0.829146, T: 481622, Avg. loss: 0.135661
Total training time: 0.19 seconds.
-- Epoch 2
Norm: 18.29, NNZs: 8252, Bias: -0.822927, T: 963244, Avg. loss: 0.117880
Total training time: 0.38 seconds.
-- Ep

[Parallel(n_jobs=1)]: Done 123 out of 123 | elapsed:  2.3min finished


['Electronics_tfidf_sgd_leaf.pkl']

# Test run

In [ ]:
print(sgd_search.classes_)
prediction = sgd_search.predict(X_test.head(10))
actual = test_set.head(10)['category'].apply(lambda catlist: catlist[LEVEL])
print('Title + Description: ', test_set.head(10)['title'],
      '\nActual category:', actual, 
      '\nPredicted:', prediction 
      )

['AV Receivers & Amplifiers' 'Accessories' 'Amplifiers'
 'Analog-to-Digital (DTV) Converters' 'Armbands'
 'Audio & Video Accessories' 'Aviation Electronics' 'Bags & Cases'
 'Bags, Cases & Sleeves' 'Barking-Dog Alarms' 'Batteries & Battery Packs'
 'Batteries & Chargers' 'Batteries, Chargers & Accessories'
 'Binoculars & Scopes' 'Biometrics' 'Blank Media' 'Block Blue Light'
 'Blu-ray Players & Recorders' 'Bluetooth & FM Transmitters' 'Boomboxes'
 'Bundles' 'CB & Two-Way Radios' 'Cables' 'Camera Cases' 'Car Electronics'
 'Cases' 'Cassette Player Adapters' 'Cassette Players & Recorders'
 'Chargers & Adapters' 'Chargers & Power Adapters'
 'Clips, Arm & Wristbands' 'Compact Radios & Stereos'
 'Computer Accessories & Peripherals' 'Computer Components'
 'Computers & Tablets' 'Cord Management' 'Covers'
 'DVD Players & Recorders' 'DVD-RW Discs' 'DVD-VCR Combos' 'Data Storage'
 'Digital Cameras' 'Digital Voice Recorders' 'Earbud Headphones'
 'Electronics' 'Film Photography' 'Fish Finders & Depth 

Pandas Apply:   0%|          | 0/10 [00:00<?, ?it/s]

Title + Description:  0                                       One Hot Summer
1    Kelby Training DVD: Adobe Photoshop CS5 Crash ...
2                                        Natural Order
3    The Runes of the Earth : The Last Chronicles o...
4    Nook Hd + 9-Inch Groovy Protective Stand Cover...
5                              Nook Vara Stylus 2-in-1
6    NOOK HD+ Protective Cover Fits 9 inch NOOK HD+...
7    Barnes & Noble Jonathan Adler Book Reader Cove...
8                      Shmirshky: think inside the box
9             Passing the Nevada Math Proficiency Exam
Name: title, dtype: object 
Actual category: 0                         eBook Readers
1    Computer Accessories & Peripherals
2                         eBook Readers
3                         eBook Readers
4                    Tablet Accessories
5                               Bundles
6                                Covers
7                                Covers
8                         eBook Readers
9                 Telep

# Evaluation
Against test set.

In [ ]:
from sklearn import metrics
from sklearn import preprocessing
y_avg = preprocessing.label_binarize(y_test,classes = sgd_search.classes_)
score = sgd_search.predict_proba(X_test)

test_output = sgd_search.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, test_output))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:", metrics.precision_score(y_test, test_output, average='weighted'))
print("Average Precision Score: ", metrics.average_precision_score(y_avg,score,average = "micro"))

Selecting column title
Stemming column title


Pandas Apply:   0%|          | 0/60202 [00:00<?, ?it/s]

Selecting column title
Stemming column title


Pandas Apply:   0%|          | 0/60202 [00:00<?, ?it/s]

Accuracy: 0.8129962459718946


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision: 0.8111343454404656
Average Precision Score:  0.8662358978185392
